In [1]:
import pandas as pd
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets, transforms

from torchmetrics.classification import Accuracy, Precision, Recall, F1Score, ConfusionMatrix


In [3]:
# Defines the device on which the processing is going to take place.
device = torch.device("cpu")
# Preprocessing takes most of the time. 
# No significant reduction in training time if we use GPU as the model is not very big.


In [4]:
class BinarizeTransform:
    """
    A class to binarize the input MNIST data.
    """
    def __call__(self, img):
        # Values are between 0 and 1 so I have binarized with threshold of 0.5
        return (img>0.5).float()

# Transform to be applied on to the data immediately after loading from location.
transform = transforms.Compose([
    transforms.ToTensor(),
    BinarizeTransform()
])

# Load and transform MNIST data
mnist_data_train = datasets.MNIST(root="data/MNIST", train=True, download=True, transform=transform)
mnist_data_test = datasets.MNIST(root="data/MNIST", train=False, download=True, transform=transform)


In [5]:
train_size = int(0.9 * len(mnist_data_train))       # Size of the train split
val_size = len(mnist_data_train) - train_size       # Size of the validation split


train_data, val_data = random_split(mnist_data_train, [train_size, val_size])




In [6]:
len(train_data), len(val_data), len(mnist_data_test)

(54000, 6000, 10000)

In [7]:
# Dictionary of all the rows of each mask-set in every file 
json_data = {
    "365nm":{
        "I1":range(0, 5),
        "I2":range(10, 15),
        "I3":range(18, 23),
        "I4":range(25, 30)
    },
    "455nm":{
        "I1":range(0, 5),
        "I2":range(7, 12),
        "I3":range(14, 19),
        "I4":range(21, 26)
    },
    "White":{
        "I1":range(0, 5),
        "I2":range(9, 14),
        "I3":range(16, 21),
        "I4":range(24, 29)
    }
}

In [16]:
filename = "365nm"  # Excel document name from which data has to be extracted.
path = "data/"+filename+".xlsx" # Excel doc path

df = pd.read_excel(path, usecols='B:Q') # Read the excel sheet
# Break the sheet down into different `DataFrame`s for every Mask-set in the table
tables = [df.iloc[json_data[filename][key]].copy().reset_index(drop=True) for key in list(json_data[filename].keys())]

NUMBER_OF_MASK_SETS = 1
mask_sets = np.random.randint(0, 20, (NUMBER_OF_MASK_SETS,))
mask_sets = np.array([(n//5, n%5) for n in mask_sets])  # [ (current_mask, optical_mask) ]

# Model hyperparameters
BATCH_SIZE = 64
EPOCHS = 100
learning_rate = 0.001

In [17]:
class CustomDataset(Dataset):
    def __init__(self, mnist_data, tables, mask_sets):

        self.processed_data = []
        self.labels = []

        for idx in range(len(mnist_data)):
            image, label = mnist_data[idx]
            image = image.reshape(1, 196, 4)
            image_combined = (
                image[:, :, 0] * 1000 + 
                image[:, :, 1] * 100 + 
                image[:, :, 2] * 10 + 
                image[:, :, 3]
            )
            x = [ tables[current_mask].loc[optical_mask][image_combined[0].tolist()].tolist()
                 for current_mask, optical_mask in mask_sets
                 ]
            x = torch.tensor(x).reshape((1, 196 * len(mask_sets)))
            self.processed_data.append(x)
            self.labels.append(label)

            if (idx+1)%1000 == 0:
                print(f"{(idx+1)}/{len(mnist_data)} done")

        
        self.processed_data = torch.cat(self.processed_data, axis = 0)
        self.labels = torch.tensor(self.labels)
                
    def __len__(self):
        return self.processed_data.shape[0]
    
    def __getitem__(self, index):
        return self.processed_data[index], self.labels[index]

In [18]:
# Load and preprocess training dataset
train_dataset = CustomDataset(train_data, tables=tables, mask_sets=mask_sets)

1000/54000 done
2000/54000 done
3000/54000 done
4000/54000 done
5000/54000 done
6000/54000 done
7000/54000 done
8000/54000 done
9000/54000 done
10000/54000 done
11000/54000 done
12000/54000 done
13000/54000 done
14000/54000 done
15000/54000 done
16000/54000 done
17000/54000 done
18000/54000 done
19000/54000 done
20000/54000 done
21000/54000 done
22000/54000 done
23000/54000 done
24000/54000 done
25000/54000 done
26000/54000 done
27000/54000 done
28000/54000 done
29000/54000 done
30000/54000 done
31000/54000 done
32000/54000 done
33000/54000 done
34000/54000 done
35000/54000 done
36000/54000 done
37000/54000 done
38000/54000 done
39000/54000 done
40000/54000 done
41000/54000 done
42000/54000 done
43000/54000 done
44000/54000 done
45000/54000 done
46000/54000 done
47000/54000 done
48000/54000 done
49000/54000 done
50000/54000 done
51000/54000 done
52000/54000 done
53000/54000 done
54000/54000 done


In [19]:
validation_dataset = CustomDataset(val_data, tables=tables, mask_sets=mask_sets)

1000/6000 done
2000/6000 done
3000/6000 done
4000/6000 done
5000/6000 done
6000/6000 done


In [20]:
test_dataset = CustomDataset(mnist_data_test, tables=tables, mask_sets=mask_sets)

1000/10000 done
2000/10000 done
3000/10000 done
4000/10000 done
5000/10000 done
6000/10000 done
7000/10000 done
8000/10000 done
9000/10000 done
10000/10000 done


In [21]:

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [22]:
class ReadoutLayer(nn.Module):
    """Readout layer.
    Parameters - 

    Attributes - 
    fc : `nn.Linear`
            Fully connected layer to be trained for reservoir computing.
    activation : `nn.functional.leaky_relu`
            Activation layer to be applied
    
    softmax : `nn.Softmax`
            Softmax activation function to get the one-hot encoded results.
    
    """
    def __init__(self):
        # super function to initialize the constructors of the parent classes.
        super(ReadoutLayer, self).__init__()
        # Class Attributes
        self.fc = nn.Linear(len(mask_sets)*196, 10) 
        self.activation = nn.functional.leaky_relu
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        """Forward method to be executed on function call.
        Parameters - 
        x : torch.tensor
            Shape( Batch_size, 196 * len(mask_sets) )
        
        Returns : torch.tensor
            Shape( Batch_size, 10 )
        """
        x = self.fc(x)
        x = self.activation(x)
        x = self.softmax(x)
        return x

In [23]:
# Model initialization
model = ReadoutLayer()
# Loss function, Categorical crossentropy loss for multi-class classification problem
criterion = nn.CrossEntropyLoss()
# Optimizer to update the gradients.
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
val_accuracy, val_precision, val_recall, val_fscore = [], [], [], []
# Evaluation metrics
accuracy = Accuracy(task="multiclass", num_classes=10).to(device)
precision = Precision(task="multiclass", num_classes=10, average='macro').to(device)
recall = Recall(task="multiclass", num_classes=10, average='macro').to(device)
f1_score = F1Score(task="multiclass", num_classes=10, average='macro').to(device)

# Class-wise Confusion matrix
confusion_matrix = ConfusionMatrix(task="multiclass", num_classes=10).to(device)

# Training loop
for epoch in range(EPOCHS):
    # Training phase
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  # Move to device
        outputs = model(images.float())  # Forward pass
        loss = criterion(outputs, labels)  # Loss calculation
        optimizer.zero_grad()
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

    print(f'Epoch [{epoch+1}/{EPOCHS}], Loss: {loss.item():.4f}')
    
    # Validation phase
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images.float())
            preds = outputs.argmax(dim=1)

            # Update metrics
            accuracy.update(preds, labels)
            precision.update(preds, labels)
            recall.update(preds, labels)
            f1_score.update(preds, labels)

        # Print validation metrics
        print(f'Validation Accuracy: {accuracy.compute().item():.4f} Precision: {precision.compute().item():.4f} ', end="")
        print(f'Validation Recall: {recall.compute().item():.4f} F1 Score: {f1_score.compute().item():.4f}')

        # Updating the list to save current metrics
        val_accuracy.append(accuracy.compute().item())
        val_precision.append(precision.compute().item())
        val_recall.append(recall.compute().item())
        val_fscore.append(f1_score.compute().item())

        # Reset metrics for the next epoch
        accuracy.reset()
        precision.reset()
        recall.reset()
        f1_score.reset()
        confusion_matrix.reset()


Epoch [1/100], Loss: 2.3007
Validation Accuracy: 0.1125 Precision: 0.0112 Validation Recall: 0.1000 F1 Score: 0.0202
Epoch [2/100], Loss: 2.3028
Validation Accuracy: 0.1125 Precision: 0.0112 Validation Recall: 0.1000 F1 Score: 0.0202
Epoch [3/100], Loss: 2.3024
Validation Accuracy: 0.1125 Precision: 0.0112 Validation Recall: 0.1000 F1 Score: 0.0202
Epoch [4/100], Loss: 2.3062
Validation Accuracy: 0.1125 Precision: 0.0112 Validation Recall: 0.1000 F1 Score: 0.0202
Epoch [5/100], Loss: 2.3026
Validation Accuracy: 0.1125 Precision: 0.0112 Validation Recall: 0.1000 F1 Score: 0.0202
Epoch [6/100], Loss: 2.2995
Validation Accuracy: 0.1125 Precision: 0.0112 Validation Recall: 0.1000 F1 Score: 0.0202
Epoch [7/100], Loss: 2.3018
Validation Accuracy: 0.1125 Precision: 0.0112 Validation Recall: 0.1000 F1 Score: 0.0202
Epoch [8/100], Loss: 2.2864
Validation Accuracy: 0.1125 Precision: 0.0112 Validation Recall: 0.1000 F1 Score: 0.0202
Epoch [9/100], Loss: 2.3045
Validation Accuracy: 0.1125 Precisio

KeyboardInterrupt: 

In [24]:
train_dataset.processed_data.shape

torch.Size([54000, 196])

In [ ]:

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        # Move images and labels to GPU
        images, labels = images.to(device), labels.to(device)

        outputs = model(torch.tensor(images, dtype=torch.float32))
        _, predicted = torch.max(outputs, 1)

        # Append predictions and labels for metric calculations
        all_preds.append(predicted)
        all_labels.append(labels)


# Concatenate all predictions and labels
all_preds = torch.cat(all_preds).to(device)
all_labels = torch.cat(all_labels).to(device)

# Calculate metrics
test_accuracy = accuracy(all_preds, all_labels)
test_precision = precision(all_preds, all_labels)
test_recall = recall(all_preds, all_labels)
test_f1 = f1_score(all_preds, all_labels)
test_confusion_matrix = confusion_matrix(all_preds, all_labels)

print(f'Test Accuracy: {test_accuracy * 100:.2f}%')
print(f'Test Precision: {test_precision*100:.4f}%')
print(f'Test Recall: {test_recall*100:.4f}%')
print(f'Test F1 Score: {test_f1:.4f}')
print("Confusion Matrix:")
print(test_confusion_matrix)

        

Test Accuracy: 87.48%
Test Precision: 87.4476%
Test Recall: 87.2869%
Test F1 Score: 0.8729
Confusion Matrix:
tensor([[ 927,    0,   12,    4,    1,    6,   12,    6,   10,    2],
        [   0, 1093,    2,   10,    2,    3,    7,    2,   16,    0],
        [  10,    8,  872,   21,   26,    7,   14,   12,   55,    7],
        [   2,    4,   32,  877,    3,   30,    3,   16,   32,   11],
        [   0,    8,    5,    1,  874,    1,   19,   11,   10,   53],
        [  13,   10,    5,   61,   24,  691,   16,   16,   48,    8],
        [  18,    4,   12,    0,   17,   15,  878,    4,    7,    3],
        [   2,   15,   15,    9,   13,    1,    0,  925,   16,   32],
        [  15,   14,   26,   27,   16,   23,   14,   18,  811,   10],
        [   7,   10,    3,   21,   88,   14,    0,   44,   22,  800]])


C:\Users\adith\AppData\Local\Temp\ipykernel_30172\247194469.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs = model(torch.tensor(images, dtype=torch.float32))


In [ ]:
save_path = "data/mnist_results/" + filename + '_' + ''.join(map(str, mask_sets)) + '.npz'
np.savez(save_path,
         predictions = all_preds,
         labels = all_labels,
         validation_accuracy = val_accuracy,
         validation_precision = val_precision,
         validation_recall = val_recall,
         validation_fscore = val_fscore
         
)

In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torchvision import datasets, transforms
# from torch.utils.data import DataLoader
# from torchmetrics.classification import Accuracy, Precision, Recall, F1Score, ConfusionMatrix

# device = torch.device("cpu")

# # Hyperparameters
# batch_size = 64
# learning_rate = 0.001
# epochs = 5

# # MNIST dataset
# transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
# train_dataset = datasets.MNIST(root='./data/MNIST', train=True, transform=transform, download=False)
# test_dataset = datasets.MNIST(root='./data/MNIST', train=False, transform=transform)

# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# # Define the single-layer MLP model
# class SingleLayerMLP(nn.Module):
#     def __init__(self):
#         super(SingleLayerMLP, self).__init__()
#         self.fc = nn.Linear(784, 10)  # Input layer (784) to output layer (10 classes)

#     def forward(self, x):
#         x = x.view(-1, 28*28)  # Flatten the input
#         x = self.fc(x)
#         return x

# # Initialize model, loss function, and optimizer
# model = SingleLayerMLP()
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# # Initialize metrics
# accuracy = Accuracy(task="multiclass", num_classes=10).to(device)
# precision = Precision(task="multiclass", num_classes=10, average='macro').to(device)
# recall = Recall(task="multiclass", num_classes=10, average='macro').to(device)
# f1_score = F1Score(task="multiclass", num_classes=10, average='macro').to(device)
# confusion_matrix = ConfusionMatrix(task="multiclass", num_classes=10).to(device)

# # Training loop
# for epoch in range(epochs):
#     model.train()
#     for images, labels in train_loader:
#         # Move images and labels to GPU
#         images, labels = images.to(device), labels.to(device)

#         # Forward pass
#         outputs = model(images)
#         loss = criterion(outputs, labels)

#         # Backward pass and optimization
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#     print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# # Testing loop
# model.eval()
# all_preds = []
# all_labels = []

# with torch.no_grad():
#     for images, labels in test_loader:
#         # Move images and labels to GPU
#         images, labels = images.to(device), labels.to(device)

#         outputs = model(images)
#         _, predicted = torch.max(outputs, 1)

#         # Append predictions and labels for metric calculations
#         all_preds.append(predicted)
#         all_labels.append(labels)

# # Concatenate all predictions and labels
# all_preds = torch.cat(all_preds)
# all_labels = torch.cat(all_labels)

# # Calculate metrics
# test_accuracy = accuracy(all_preds, all_labels)
# test_precision = precision(all_preds, all_labels)
# test_recall = recall(all_preds, all_labels)
# test_f1 = f1_score(all_preds, all_labels)
# test_confusion_matrix = confusion_matrix(all_preds, all_labels)

# print(f'Test Accuracy: {test_accuracy * 100:.2f}%')
# print(f'Test Precision: {test_precision*100:.4f}%')
# print(f'Test Recall: {test_recall*100:.4f}%')
# print(f'Test F1 Score: {test_f1:.4f}')
# print("Confusion Matrix:")
# print(test_confusion_matrix)